In [1]:
INPUT_TRAIN = "../input/train.csv"
INPUT_TEST = "../input/test.csv"

TRAIN='train-processed.csv'
TEST='test-processed.csv'
Y='target.csv'

In [10]:
import os
import gc
import json
import time
from datetime import datetime
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

import warnings
warnings.filterwarnings('ignore')

In [14]:
def load_df(csv_path='./input/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, coverters = {column: json.loads for column in JSON_COLUMNS}, 
                     dtype = {'fullVisitorId': 'str'},
                     nrows=nrows)
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

print(os.listdir("./input"))
                     

['.DS_Store', 'sample_submission_v2.csv', 'train_v2.csv.zip', 'sample_submission_v2.csv.zip']


In [18]:
def process_dfs(train_df, test_df):
    print("Processing dfs...")
    print("Dropping repeated columns")
    columns = [col for col in train_df.columns if train_df[col].nunique() > 1]
    
    train_df = train_df[columns]
    test_df = test_df[columns]
    
    trn_len = train_df.shape[0]
    merged_df = pd.concat([train_df, test_df])
    merged_df['diff_visitId_time'] = merged_df['visitId'] - merged_df['visitStartTime']
    merged_df['diff_visitId_time'] = (merged_df['diff_visitId_time'] != 0).astype(int)
    del merged_df['visitId']

    del merged_df['sessionId']

    print("Generating date columns...")
    format_str = '%Y%m%d' 
    
    